<a href="https://colab.research.google.com/github/mkazachkin/LearningTransfer/blob/main/HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание

Реализуйте классификацию эмоциональной окрашенности текстов при помощи объекта pipeline.

1. Найдите тип задач Sentiment Analysis на huggingface.co
2. Найдите модель для русского языка (примеры: rubert-tiny2..., rubert-base...)

In [53]:
import pandas as pd
import warnings

from transformers import pipeline

warnings.filterwarnings('ignore')

def get_label(predictions):
    label = predictions[0]['label']
    score = predictions[0]['score']
    return label, round(score, 2)

def classify_text(text, labels, pipe):
    predictions = pipe([text])
    label, score = get_label(predictions)
    return text, labels[label], score

In [54]:
# Определение моделей и меток
MODELS = {
    'seara/rubert-base-cased-ru-go-emotions': {
        'admiration': 'восхищение', 'amusement': 'веселье',
        'anger': 'злость', 'annoyance': 'раздражение',
        'approval': 'одобрение', 'caring': 'забота',
        'confusion': 'непонимание', 'curiosity': 'любопытство',
        'desire': 'желание', 'disappointment': 'разочарование',
        'disapproval': 'неодобрение', 'disgust': 'отвращение',
        'embarrassment': 'смущение', 'excitement': 'возбуждение',
        'fear': 'страх', 'gratitude': 'признательность', 'grief': 'горе',
        'joy': 'радость', 'love': 'любовь', 'nervousness': 'нервозность',
        'optimism': 'оптимизм', 'pride': 'гордость', 'realization': 'осознание',
        'relief': 'облегчение', 'remorse': 'раскаяние',
        'sadness': 'грусть', 'surprise': 'удивление', 'neutral': 'нейтральность',
    },
    'seara/rubert-tiny2-russian-sentiment': {
        'neutral': 'нейтральный',
        'positive': 'позитивный',
        'negative': 'негативный',
    },
    'blanchefort/rubert-base-cased-sentiment': {
        'NEUTRAL': 'нейтральный',
        'POSITIVE': 'позитивный',
        'NEGATIVE': 'негативный',
    },
}


In [55]:
TEXT_LINES = ["Отличный товар",
              "Мне не понравилось",
              "Прекрасно выглядит, ничего не меняйте!",
              "Ничего не понял",
              "У меня не работала розетка, в остальном все отлично"]

In [57]:
results = {text: [] for text in TEXT_LINES}

for model_name, labels in MODELS.items():
    pipe = pipeline("text-classification", model=model_name)
    for text in TEXT_LINES:
        try:
            _, label, score = classify_text(text, labels, pipe)
            results[text].append(f"{label} ({score})")
        except Exception as e:
            results[text].append(f"Error ({str(e)})")


In [58]:
df = pd.DataFrame.from_dict(
    results,
    orient='index',
    columns=[model for model in MODELS.keys()]
    )
df.reset_index(inplace=True)
df.columns = ['Text'] + [f'Model {i+1} Label (Score)' for i in range(len(MODELS))]

styled_df = df.style.set_properties(**{'text-align': 'left'})
styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
styled_df

,Text,Model 1 Label (Score),Model 2 Label (Score),Model 3 Label (Score)
0,Отличный товар,восхищение (0.92),позитивный (0.99),позитивный (0.98)
1,Мне не понравилось,нейтральность (0.41),нейтральный (0.59),нейтральный (0.62)
2,"Прекрасно выглядит, ничего не меняйте!",восхищение (0.94),позитивный (0.94),негативный (0.75)
3,Ничего не понял,нейтральность (0.42),негативный (0.52),нейтральный (0.82)
4,"У меня не работала розетка, в остальном все отлично",веселье (0.4),позитивный (0.67),позитивный (0.98)
